In [115]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
import threading
from queue import Queue
import pandas as pd

In [102]:
# Função para pegar os links do botão detalhar
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

# Função para acessar o portal com os parametros desejados
def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

# Função para verificar se o número da página é maior que o valor anteriormente armazenado
def page_number_greater(driver, previous_page):
    page_text = driver.find_element(By.ID, 'lista_info').text
    current_page = int(page_text.split()[1])
    return current_page > previous_page

In [ ]:
beneficio = 'bolsa-familia'
start_date = '01/01/2020'
end_date = '31/12/2020'
municipio = 'Tanguá'

# Iniciando drive do Chorme
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver, beneficio, start_date, end_date, municipio)
sleep(5)

# Clica no botão de ver paginação completa
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
sleep(5)

# Seleciona a opção de 50 individuos por páginas
elemento_dropdown  = driver.find_element(By.NAME, 'lista_length')
Select(elemento_dropdown).select_by_value('50')
sleep(5)

# Verifica quantas paginas no total
quantPaginas = driver.find_element(By.ID, 'lista_info').text.split()[-1]

# Crie uma lista para armazenar os valores de atributo href
hrefs = []
previous_page = 1
for _ in range(int(quantPaginas)):
    # Busca os lnkes do detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Tenta obeter o link, caso a requisição demore para responder, espere mais 5 segundos
    try:
        link = get_href(elements)
    except:
        sleep(5)
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        link = get_href(elements)


    # Adiciona na lista de links
    hrefs.extend(link)

    # Clica no botão proximo  
    driver.find_element(By.ID, 'lista_next').click()

    # Aguardar a mudança de página
    WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page))
    previous_page += 1

driver.quit()

## Tratamento dos Hrefs

In [39]:
df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

In [40]:
ano = start_date.split('/')[-1]

df.to_csv(f'{start_date}.csv')

In [105]:
df = pd.read_csv('2017.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv('bolsa-familia-2017.csv')

# Testando Threads

In [4]:
def getQuantPaginas():
    driver = webdriver.Chrome()
    consultar(driver,'bolsa-familia', '01/01/2016', '31/12/2016', 'Tanguá')
    # inicia a pagian e espera 10 segundos para os dados carregarem
    sleep(5)

    # Clica no botão de ver paginação completa
    # driver.find_element(By.XPATH, '//*[@id="lista_wrapper"]/div/div[3]/div[3]/button').click()
    driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()



    # Clica no select para aumentar o numero de dados na pagina
    elemento_dropdown  = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(5)
    # Verifica quantas paginas possuem
    quantPaginas = driver.find_element(By.XPATH, '//*[@id="lista_info"]').text.split()[-1]
    driver.quit()
    return int(quantPaginas)

In [ ]:
# Função que será executada em cada thread
def coletar_dados(start_page, end_page, beneficio, start_date, end_date, municipio='Tanguá'):
    driver = webdriver.Chrome()
    driver.execute_script("window.open('about:blank', 'new_tab');")
    driver.switch_to.window(driver.window_handles[-1])
    consultar(driver, beneficio, start_date, end_date, municipio)
    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'botao__gera_paginacao_completa'))
        )
    finally:
        driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()

    sleep(5)
    elemento_dropdown = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(6)
    driver.find_element(By.ID, 'paginas-selecao-1-lista').send_keys(start_page)
    driver.find_element(By.ID, 'botao-ir-para-a-pagina-lista').click()
    sleep(7)
    hrefs = []
    for _ in range(start_page, end_page + 1):
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        hrefs.extend(get_href(elements))
        driver.find_element(By.ID, 'lista_next').click()
        sleep(10)

    driver.quit()

    if results_queue:
        results_queue.put(hrefs)


# Execução das threads
beneficio = 'bolsa-familia'
start_date = '01/01/2018'
end_date = '31/12/2018'
municipio = 'Tanguá'

num_threads = 6
quantPaginas = getQuantPaginas()
pages_per_thread = quantPaginas // num_threads

threads = []
results_queue = Queue()

for i in range(num_threads):
    start_page = i * pages_per_thread + 1
    end_page = start_page + pages_per_thread - 1 if i < num_threads - 1 else quantPaginas
    thread = threading.Thread(target=coletar_dados, args=(start_page, end_page, beneficio, start_date, end_date, municipio))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Recupera os resultados das threads
hrefs = []
while not results_queue.empty():
    hrefs.extend(results_queue.get())


# Teste de extrair quantidade da tabela

In [15]:
driver = webdriver.Chrome()

driver.get('https://portaldatransparencia.gov.br/beneficios/bolsa-familia/293703902?ordenarPor=mesReferencia&direcao=desc')
sleep(3)
# Localize a tabela
tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos')

# Obtenha todas as linhas da tabela
linhas = tabela.find_elements(By.TAG_NAME, 'tr')


dados = []

# Itere sobre as linhas
for linha in linhas:
    # Obtenha todas as células da linha
    celulas = linha.find_elements(By.TAG_NAME, 'td')
    
    # Extraia os dados das células
    dados_linha = [celula.text for celula in celulas]
    
    # Armazene os dados
    dados.append(dados_linha)

# Feche o navegador
# driver.quit()

# Faça algo com os dados coletados
print(dados)


[[], ['10/2021', '10/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['09/2021', '09/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['08/2021', '08/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['07/2021', '07/2021', 'RJ', 'TANGUÁ', '1', '147,00'], ['06/2021', '06/2021', 'RJ', 'TANGUÁ', '1', '147,00'], ['05/2021', '05/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['04/2021', '04/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['03/2021', '03/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['02/2021', '02/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['01/2021', '01/2021', 'RJ', 'TANGUÁ', '2', '171,00'], ['12/2020', '12/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['11/2020', '11/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['10/2020', '10/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['09/2020', '09/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['08/2020', '08/2020', 'RJ', 'TANGUÁ', '2', '178,00']]


In [16]:
dados

[[],
 ['10/2021', '10/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['09/2021', '09/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['08/2021', '08/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['07/2021', '07/2021', 'RJ', 'TANGUÁ', '1', '147,00'],
 ['06/2021', '06/2021', 'RJ', 'TANGUÁ', '1', '147,00'],
 ['05/2021', '05/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['04/2021', '04/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['03/2021', '03/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['02/2021', '02/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['01/2021', '01/2021', 'RJ', 'TANGUÁ', '2', '171,00'],
 ['12/2020', '12/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['11/2020', '11/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['10/2020', '10/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['09/2020', '09/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['08/2020', '08/2020', 'RJ', 'TANGUÁ', '2', '178,00']]

In [14]:
len(linhas)

16